# Scripts to build
* Extract values from Atlas
* Convert to probabilities
* Prepare inputs after masking
* Generate results

In [1]:
import os
import pandas
import numpy as np
import nibabel as ni
from glob import glob
from nilearn import image

### Testing Extract Values from Atlas

In [65]:
files_in = '/home/users/jvogel/Science/ADNI_tau/template_space/tau_images/smoothed_wscored_scans/'
atlas = '/home/users/jvogel/Science/templates/atlases/Lund/hippocampus_labels_LUND_scif55_1p5.nii.gz'


In [66]:
%%time
df = Extract_Values_from_Atlas(files_in,atlas)

It seems you passed a directory
found 238 images!
working on subject 0
working on subject 1
working on subject 2
working on subject 3
working on subject 4
working on subject 5
working on subject 6
working on subject 7
working on subject 8
working on subject 9
working on subject 10
working on subject 11
working on subject 12
working on subject 13
working on subject 14
working on subject 15
working on subject 16
working on subject 17
working on subject 18
working on subject 19
working on subject 20
working on subject 21
working on subject 22
working on subject 23
working on subject 24
working on subject 25
working on subject 26
working on subject 27
working on subject 28
working on subject 29
working on subject 30
working on subject 31
working on subject 32
working on subject 33
working on subject 34
working on subject 35
working on subject 36
working on subject 37
working on subject 38
working on subject 39
working on subject 40
working on subject 41
working on subject 42
working on sub

In [69]:
%%time
df = Extract_Values_from_Atlas(files_in,atlas,blocking='all_at_once')

It seems you passed a directory
found 238 images!
extracting values from atlas
CPU times: user 9.42 s, sys: 13.2 s, total: 22.6 s
Wall time: 1min 38s


In [95]:
%%time
df = Extract_Values_from_Atlas(files_in,atlas,blocking=100)

It seems you passed a directory
found 238 images!
working on batch 1 of 100 subjects
processing 100 subjects
working on batch 2 of 100 subjects
processing 100 subjects
working on final batch of subjects
processing 38 subjects
CPU times: user 9.38 s, sys: 6.43 s, total: 15.8 s
Wall time: 57.6 s


In [96]:
%%time
df = Extract_Values_from_Atlas(files_in,atlas,blocking=50)

It seems you passed a directory
found 238 images!
working on batch 1 of 50 subjects
processing 50 subjects
working on batch 2 of 50 subjects
processing 50 subjects
working on batch 3 of 50 subjects
processing 50 subjects
working on batch 4 of 50 subjects
processing 50 subjects
working on final batch of subjects
processing 38 subjects
CPU times: user 9.52 s, sys: 4.38 s, total: 13.9 s
Wall time: 46.7 s


In [101]:
files_in = glob('/home/users/jvogel/Science/ADNI_tau/template_space/tau_images/smoothed_wscored_scans/*')[:10]
atlas = '/home/users/jvogel/Science/templates/atlases/Lund/hippocampus_labels_LUND_scif55_1p5.nii.gz'
labels = ['off_target','early_NFT']
df = Extract_Values_from_Atlas(files_in,atlas,blocking='all_at_once', labels=labels)


processing 10 subjects
extracting values from atlas


In [102]:
df

,off_target,early_NFT
0,-1.04061,-1.5576
1,-1.17887,-2.29193
2,-0.495905,-0.439328
3,-1.25555,-1.61055
4,-0.457695,-0.52681
5,-0.830459,-1.1219
6,-0.867813,-1.97659
7,-1.18883,-1.72155
8,-0.706084,-1.03505
9,-1.16094,-2.35611


In [111]:
files_in = glob('/home/users/jvogel/Science/ADNI_tau/template_space/tau_images/smoothed_wscored_scans/*')[:10]
atlas = '/home/users/jvogel/Science/templates/atlases/Lund/hippocampus_labels_LUND_scif55_1p5.nii.gz'
labels = ['off_target','early_NFT']
sids = ['sub_%s'%x for x in range(10)]
df2 = Extract_Values_from_Atlas(files_in,atlas,blocking='one_at_a_time', labels=labels, sids = sids)


processing 10 subjects
working on subject sub_0
working on subject sub_1
working on subject sub_2
working on subject sub_3
working on subject sub_4
working on subject sub_5
working on subject sub_6
working on subject sub_7
working on subject sub_8
working on subject sub_9


In [112]:
df2

,off_target,early_NFT
sub_0,-1.04061,-1.5576
sub_1,-1.17887,-2.29193
sub_2,-0.495905,-0.439328
sub_3,-1.25555,-1.61055
sub_4,-0.457695,-0.52681
sub_5,-0.830459,-1.1219
sub_6,-0.867813,-1.97659
sub_7,-1.18883,-1.72155
sub_8,-0.706084,-1.03505
sub_9,-1.16094,-2.35611


In [104]:
import os
import pandas
import numpy as np
import nibabel as ni
from glob import glob

def Extract_Values_from_Atlas(files_in, atlas, output = None, 
                              blocking = 'one_at_a_time', 
                              labels = [], sids = []):
    '''
    This function will extract mean values from a set of images for 
    each ROI from a given atlas. Returns a Subject x ROI pandas
    DataFrame (and csv file if output argument is set to a path).
    
    Use blocking argument according to memory capacity of your
    computer vis-a-vis memory requirements of loading all images.
    
    files_in: determines which images to extract values from. Input 
    can be any of the following:
        -- a list of paths
        -- a path to a directory containing ONLY files to extract from
        -- a search string (with wild card) that would return all
        desired images. For example, doing ls [files_in] in a terminal 
        would list all desired subjects
        -- a 4D Nifti image
        **NOTE** be aware of the order of file input, which relates to 
        other arguments
        
    atlas: Path to an atlas, or a Nifti image or np.ndarry of desired 
    atlas. Or, if doing native space analysis, instead, supply a list 
    of paths to atlases that match each subject. 
        NOTE: In this case, The order of this list should be the same
        order as subjects in files_in
        
    output: if you wish the resulting ROI values to be written to file,
    provide a FULL path. Otherwise, leave as None (matrix will be 
    returned)
    
    blocking: loading all images to memory at once may not be possible
    depending on your computer. Acceptable arguments are:
        -- 'one_at_a_time': will extract values from each image
        independently. Recommended for memories with poor memory 
        capacity. Required for native space extraction.
        -- 'all_at_once': loads all images into memory at once.
        Provides a slight speed up for faster machines overe
        one_at_a_time, but is probably not faster than batching (see
        below). Only recommended for smaller datasets.
        ** WARNING ** Not recommended on very large datasets. Will
        crash computers with poor memory capacity.
        -- any integer: determines the number of images to be read to
        memory at once. Recommended for large datasets. 
    
    labels: a list of string labels that represent the names of the
    ROIs from atlas. 
        NOTE: ROIs are read consecutively from lowest to highest, and
        labels *must* match that order
    Default argument [] will use "ROI_x" for each ROI, where X
    corresponds to the actual ROI integer lael
    
    sids: a list of subject IDs in the same order as files_in. Default 
    argument [] will list subjects with consecutive integers.
    
    '''
    
    if type(blocking) == str and blocking not in ['all_at_once','one_at_a_time']:
        raise IOError('blocking only accepts integers or argumennts of "all_at_once" or "one_at_a_time"')
    
    if type(atlas) == list: 
        if blocking != 'one_at_a_time':
            print('WARNING: you have passed a list of atlases but blocking is not set to one_at_a_time')
            print('Lists of atlases are for native space situations where each subject has their own atlas')
            print('If you want to test multiple atlases, run the script multiple times with different atlases')
            raise IOError('you have passed a list of atlases but blocking is not set to one_at_a_time')
    
    if type(atlas) != list:
        if type(atlas) == str:
            try:
                atl = ni.load(atlas).get_data()
            except:
                raise IOError('could not find an atlas at the specified location: %s'%atlas)
        elif type(atlas) == ni.nifti1.Nifti1Image:
            atl = atlas.get_data()
        elif type(atlas) == np.ndarray:
            atl = atlas
        else:
            print('could not recognize atlas filetype. Please provide a path, a NiftiImage object, or an numpy ndarray')
            raise IOError('atlas type not recognized')
        
        if blocking == 'all_at_once':
            i4d = load_data(files_in, return_images=True)
            if i4d.shape[:-1] != atl.shape:
                raise IOError('image dimensions do not match atlas dimensions')
            if len(sids) == 0:
                sids = range(i4d.shape[-1])
            print('extracting values from atlas')
            roi_vals = generate_matrix_from_atlas(i4d.get_data(), atl, labels, sids)
        else:
            image_paths = load_data(files_in, return_images = False)
            if blocking == 'one_at_a_time':
                catch = []
                for i,image_path in enumerate(image_paths):
                    if len(sids) > 0: 
                        sid = [sids[i]]
                    else:
                        sid = [i]
                    print('working on subject %s'%sid[0])
                    img = ni.load(image_path).get_data()
                    try:
                        assert img.shape == atl.shape, 'fail'
                    except:
                        print('dimensions for subject %s (%s) image did not match atlas dimensions (%s)'%(sid,
                                                                                 img.shape,
                                                                                 atl.shape))
                        print('skipping subject %s'%sid[0])
                        continue
                    f_mat = generate_matrix_from_atlas(img, atl, labels, sid)
                    catch.append(f_mat)
                roi_vals = pandas.concat(catch)
            elif type(blocking) == int:
                block_size = blocking
                if len(image_paths)%block_size == 0:
                    blocks = int(len(image_paths)/block_size)
                    remainder = False
                else:
                    blocks = int((len(image_paths)/blocking) + 1)
                    remainder = True
                catch = []
                count = 0
                for block in range(blocks):
                    if block == (blocks - 1) and remainder:
                        print('working on final batch of subjects')
                        sub_block = image_paths[count:]
                    else:
                        print('working on batch %s of %s subjects'%((block+1),block_size))
                        sub_block = image_paths[count:(count+block_size)]
                    i4d = load_data(sub_block, return_images = True)
                    if i4d.shape[:-1] != atl.shape:
                        raise IOError('image dimensions (%s) do not match atlas dimensions (%)'%(atl.shape,
                                                                                                i4d.shape[:-1]
                                                                                                ))
                    if len(sids) == 0:
                        sids_in = range(i4d.shape[-1])
                    elif block == (blocks - 1) and remainder:
                        sids_in = sids[count:]
                    else:
                        sids_in = sids[count:(count+block_size)]
                    f_mat = generate_matrix_from_atlas(i4d.get_data(), atl, labels, sids_in)
                    catch.append(f_mat)
                    count += block_size
                roi_vals = pandas.concat(catch)
    else:
        image_paths = load_data(files_in, return_images = False)
        if len(atlas) != len(image_paths):
            raise IOError('number of images (%s) does not match number of atlases (%s)'%(len(image_paths),
                                                                                      len(atlas)))
        catch = []
        for i,image_path in enumerate(image_paths):
            if len(sids) > 0: 
                sid = [i]
            else:
                sid = [sids[i]]
            print('working on subject'%sid)
            img = ni.load(image_path).get_data()
            atl = ni.load(atlas[i]).get_data()
            try:
                assert img.shape == atl.shape, 'fail'
            except:
                print('dimensions for subject %s (%s) image did not match atlas dimensions (%s)'%(sid,
                                                                                                 img.shape,
                                                                                                 atl.shape
                                                                                                 ))
                print('skipping subject %s'%sid)
                continue
            f_mat = generate_matrix_from_atlas(img, atl, labels, sid)
            catch.append(f_mat)
        roi_vals = pandas.concat(catch)    

    if output:
        roi_vals.to_csv(output)
    return roi_vals
    
def generate_matrix_from_atlas(files_in, atl, labels, sids):
    
    if len(files_in.shape) == 3:
        x,y,z = files_in.shape
        files_in = files_in.reshape(x,y,z,1)
    atl = atl.astype(int)
    if max(np.unique(atl)) != (len(np.unique(atl)) -1):
        new_atl = np.zeros_like(atl)
        atl_map = dict(zip(np.unique(atl),
                           range(len(np.unique(atl)))
                          ))
        for u in np.unique(atl):
            new_atl[atl == u] = atl_map[u]
        atl = new_atl
    if len(labels) > 0:
        cols = labels
    else:
        cols = ['roi_%s'%x for x in np.unique(atl) if x != 0]
    f_mat = pandas.DataFrame(index = sids,
                             columns = cols)
    tot = np.bincount(atl.flat)
    for sub in range(files_in.shape[-1]):
        mtx = files_in[:,:,:,sub]
        sums = np.bincount(atl.flat, weights = mtx.flat)
        rois = (sums/tot)[1:]
        f_mat.loc[f_mat.index[sub]] = rois
    
    return f_mat
    

def load_data(files_in, return_images):
    
    fail = False
    
    if type(files_in) == str:
        if os.path.isdir(files_in):
            print('It seems you passed a directory')
            search = os.path.join(files_in,'*')
            flz = glob(search)
            num_f = len(flz)
            if num_f == 0:
                raise IOError('specified directory did not contain any files')
            else:
                print('found %s images!'%num_f)
            if return_images:
                i4d = ni.concat_images(flz)
        elif '*' in files_in:
            print('It seems you passed a search string')
            flz = glob(files_in)
            num_f = len(flz)
            if num_f == 0:
                raise IOError('specified search string did not result in any files')
            else:
                print('found %s images'%num_f)
            if return_images:
                i4d = ni.concat_images(flz)
        else:
            fail = True
    elif type(files_in) == list:
        flz = files_in
        print('processing %s subjects'%len(files_in))
        if return_images:
            i4d = ni.concat_images(files_in)
    elif type(files_in) == ni.nifti1.Nifti1Image:
        print('processing %s subjects'%files_in.shape[-1])
        i4d = files_in
    else:
        fail = True
        
    if fail:
        print('files_in not recognized.', 
                    'Please enter a search string, valid directory, list of paths, or a Nifti object')
        raise ValueError('I do not recognize the files_in input.')
    
    if return_images:
        return i4d
    else:
        return flz

In [89]:
files_in = '/home/users/jvogel/Science/ADNI_tau/template_space/tau_images/smoothed_wscored_scans/'
atlas = '/home/users/jvogel/Science/templates/atlases/Lund/hippocampus_labels_LUND_scif55_1p5.nii.gz'
output = None
blocking = 'one_at_a_time'
labels = []
sids = []


if type(blocking) == str and blocking not in ['all_at_once','one_at_a_time']:
    raise IOError('blocking only accepts integers or argumennts of "all_at_once" or "one_at_a_time"')

if type(atlas) == list: 
    if blocking != 'one_at_a_time':
        print('WARNING: you have passed a list of atlases but blocking is not set to one_at_a_time')
        print('Lists of atlases are for native space situations where each subject has their own atlas')
        print('If you want to test multiple atlases, run the script multiple times with different atlases')
        raise IOError('you have passed a list of atlases but blocking is not set to one_at_a_time')

if type(atlas) != list:
    if type(atlas) == str:
        try:
            atl = ni.load(atlas).get_data()
        except:
            raise IOError('could not find an atlas at the specified location: %s'%atlas)
    elif type(atlas) == ni.nifti1.Nifti1Image:
        atl = atlas.get_data()
    elif type(atlas) == np.ndarray:
        atl = atlas
    else:
        print('could not recognize atlas filetype. Please provide a path, a NiftiImage object, or an numpy ndarray')
        raise IOError('atlas type not recognized')

    if blocking == 'all_at_once':
        i4d = load_data(files_in, return_images=True)
        if i4d.shape[:-1] != atl.shape:
            raise IOError('image dimensions do not match atlas dimensions')
        if len(sids) == 0:
            sids = range(i4d.shape[-1])
        roi_vals = generate_matrix_from_atlas(i4d, atl, labels, sids)
    else:
        image_paths = load_data(files_in, return_images = False)
        


It seems you passed a directory
found 238 images!


In [90]:
catch = []
i = 0
image_path = image_paths[i]

if len(sids) > 0: 
    sid = [sids[i]]
else:
    sid = [i]
print('working on subject %s'%sid[0])
img = ni.load(image_path).get_data()


working on subject 0


In [92]:
blocking = 100
block_size = blocking
if len(image_paths)%block_size == 0:
    blocks = len(image_paths)/block_size
    remainder = False
else:
    blocks = int((len(image_paths)/blocking) + 1)
    remainder = True
catch = []
count = 0
for block in range(blocks):
    if block == (blocks - 1) and remainder:
        print('working on final batch of subjects')
        sub_block = image_paths[count:]
    else:
        print('working on batch %s of %s subjects'%((block+1),block_size))
        sub_block = image_paths[count:(count+block_size)]
    i4d = load_data(sub_block, return_images = True)
    if i4d.shape[:-1] != atl.shape:
        raise IOError('image dimensions (%s) do not match atlas dimensions (%)'%(atl.shape,
                                                                                i4d.shape[:-1]
                                                                                ))
    if len(sids) == 0:
        sids_in = range(i4d.shape[-1])
    elif block == (blocks - 1) and remainder:
        sids_in = sids[count:]
    else:
        sids_in = sids[count:(count+block_size)]
    break

working on batch 1 of 100 subjects
processing 100 subjects


In [93]:
sids_in

range(0, 100)

In [48]:
np.unique(new_atl)

array([0])

In [43]:
atl_map = dict(zip(np.unique(atl),
                       range(len(np.unique(atl)))
                      ))
atl_map

{0: 0, 1: 1, 3: 2}

In [55]:
new_atl[atl == 1]

array([0, 0, 0, ..., 0, 0, 0])

In [75]:
image_paths[0:100]

['/home/users/jvogel/Science/ADNI_tau/template_space/tau_images/smoothed_wscored_scans/002_S_0413_v1_smooth_AV1451_wscored_SUVR.nii',
 '/home/users/jvogel/Science/ADNI_tau/template_space/tau_images/smoothed_wscored_scans/041_S_5078_v1_smooth_AV1451_wscored_SUVR.nii',
 '/home/users/jvogel/Science/ADNI_tau/template_space/tau_images/smoothed_wscored_scans/016_S_4952_v1_smooth_AV1451_wscored_SUVR.nii',
 '/home/users/jvogel/Science/ADNI_tau/template_space/tau_images/smoothed_wscored_scans/002_S_5178_v1_smooth_AV1451_wscored_SUVR.nii',
 '/home/users/jvogel/Science/ADNI_tau/template_space/tau_images/smoothed_wscored_scans/135_S_5269_v1_smooth_AV1451_wscored_SUVR.nii',
 '/home/users/jvogel/Science/ADNI_tau/template_space/tau_images/smoothed_wscored_scans/041_S_4874_v1_smooth_AV1451_wscored_SUVR.nii',
 '/home/users/jvogel/Science/ADNI_tau/template_space/tau_images/smoothed_wscored_scans/002_S_1261_v1_smooth_AV1451_wscored_SUVR.nii',
 '/home/users/jvogel/Science/ADNI_tau/template_space/tau_image